In [31]:
import numpy as np


def entropy(examples):
    """計算熵。"""
    counts = np.unique(examples[:, -1], return_counts=True)
    p = counts[1] / len(examples)
    return -p * np.log2(p)


def importance(attribute, examples):
    """計算屬性的重要性。"""
    left_examples = examples[examples[:, attribute] == 0]
    right_examples = examples[examples[:, attribute] == 1]
    return entropy(examples) - (entropy(left_examples) + entropy(right_examples)) / 2


def decision_tree_learning(examples, attributes, parent_examples):
    """決策樹學習。"""
    if len(examples) == 0:
        return parent_examples.mode(axis=0)[0]
    elif all(examples[:, -1] == examples[0, -1]):
        return examples[:, -1].item()
    elif len(attributes) == 0:
        return parent_examples.mode(axis=0)[0]
    else:
        attribute = attributes[np.argmax(
            [importance(a, examples) for a in attributes])]
        print(attribute)
        tree = DecisionTree(attribute)
        for value in np.unique(examples[:, attribute]):
            exs = examples[examples[:, attribute] == value]
            # subtree = decision_tree_learning(exs, attributes - [attribute], examples)
            subtree = decision_tree_learning(exs, [attr for attr in attributes if attr != attribute], examples)
            tree.add_branch(value, subtree)
        return tree


class DecisionTree(object):
    """決策樹類別。"""

    def __init__(self, attribute):
        self.attribute = attribute
        self.left = None
        self.right = None

    def add_branch(self, value, subtree):
        self.left = DecisionTree(value) if value < self.attribute else None
        self.right = subtree

    def classify(self, example):
        if example[self.attribute] < self.attribute:
            if self.left is not None:
                print("left ", self.left.classify(example))
                return self.left.classify(example)
            else:
                return None
        else:
            print("right ", self.right.classify(example))
            return self.right.classify(example)

In [33]:
# 生成數據集
examples = np.array([
    [0,0,0,0],
    [0,0,0,1],
    [0,1,0,1],
    [0,1,1,0],
    [0,0,0,0],
    [1,0,0,0],
    [1,0,0,1],
    [1,1,1,1],
    ])
attributes = [0, 1, 2]
parent_examples = None
# 訓練決策樹
tree = decision_tree_learning(examples, attributes, parent_examples)

# 預測結果
print(tree.classify([0, 0, 0]))
# 輸出：0

1


ValueError: operands could not be broadcast together with shapes (2,) (0,) 